In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split



In [6]:
df = pd.read_csv('pbta-gene-counts-rsem-expected_count-collapsed.combined.filtered.target.highlow.T.zip')

In [7]:
df.shape

(415, 19249)

In [8]:
df.head()

,Unnamed: 0,MT-CO1,GFAP,MT-ND4,MT-CO3,MT-CO2,MT-ATP6,EEF1A1,MT-ND2,MT-CYB,MT-ND5,MT-ND1,MBP,CLU,ACTB,SPARC,ACTG1,VIM,GAPDH,PLP1,TUBA1A,MT-ND3,EEF2,MT-ND6,FTH1,PEA15,APOE,AQP4,MT-ND4L,BCAN,B2M,FTL,SPP1,TPT1,PSAP,RPL3,PTPRZ1,GNAS,RPLP0,TMSB4X,...,LYZL6,OR6Q1,PRAMEF13,AL353572.3,FAM236C,AC233724.17,AC018793.3,AC106774.8,AC093762.1,OR8B2,AL132671.2,AC233724.19,AC106774.6,AC233724.20,AC233724.21,USP17L13,USP17L17,AC092017.3,AC106774.9,AC005833.1,AC106774.7,OR4F4,PRAMEF6,DEFB103A,DEFB103B,PRAMEF9,MBD3L4,TSPY3,AC018793.2,AC018793.4,AC104581.3,BPY2,BPY2B,BPY2C,PRAMEF5,AC106774.5,AC083800.1,AC118470.1,target,HighLowGrade
0,BS_0VXZCRJS,688399.97,768375.44,489755.90,240854.75,170172.52,216031.63,484337.54,126346.58,213623.0,174158.0,86484.21,3068.00,188312.0,286578.16,462590.00,258658.00,349459.75,140807.81,9206.0,143135.53,46123.0,131106.0,55004.0,72654.06,94949.78,94892.0,13438.0,46410.0,31219.83,102763.0,39143.0,105561.0,88778.0,157278.0,86229.66,237981.0,102867.28,59580.95,63591.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1.0,1
1,BS_3AC3SRWH,6885942.15,695792.99,3068246.64,2359968.34,1788848.49,1772846.91,145654.84,2186492.93,1838574.0,1889361.0,1820432.80,3673422.60,14560.0,19284.02,4628.00,9892.98,1845.80,21470.67,246679.0,11360.89,433865.0,23476.0,676573.0,112014.72,33071.98,8012.0,3608.0,269167.0,2781.91,4241.0,44398.0,2957.0,44718.0,3553.0,26153.28,1195.0,13022.00,6597.45,53926.97,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.72,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1.0,1
2,BS_4PPHAQXF,1042630.21,249792.48,651345.95,390247.42,272135.79,295352.62,628701.58,200263.92,253190.0,288914.0,181970.07,1445.64,313007.0,230195.17,266144.00,146740.16,123370.53,136781.80,3583.0,53928.84,57121.0,126974.0,87469.0,49422.88,121362.88,241669.0,127996.0,62207.0,47788.93,83203.0,29494.0,73165.0,93585.0,58850.0,70306.64,418586.0,101705.48,50452.96,97692.61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1
3,BS_4PWDGEB0,914337.23,17581.45,588803.00,349555.99,201104.00,274655.78,305689.05,269700.40,269897.0,261736.0,239868.11,809.00,747077.0,217459.08,252539.99,188274.00,227649.45,448264.28,1980.0,69938.10,59566.0,77910.0,84515.0,110334.77,102432.82,12124.0,42731.0,58580.0,4284.22,54555.0,39668.0,5633.0,38206.0,66213.0,52877.54,39382.0,112132.86,20984.12,55460.98,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.69,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1.0,1
4,BS_58YXHGAJ,617360.80,477984.96,398712.00,248921.83,190080.20,198601.08,483424.31,186781.16,189343.0,131913.0,150331.86,105316.27,113452.0,171486.03,24650.99,109645.00,13476.74,75776.85,87503.0,157220.62,45468.0,123196.0,55188.0,54576.94,87553.81,94724.0,47805.0,29622.0,115295.59,39423.0,40896.0,9999.0,64869.0,49058.0,78473.83,68163.0,51384.51,68780.96,71239.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.33,0.0,0.0,0.0,0.0,1


In [9]:
train_cols = df.columns[1:-2]
label = df.columns[-1]
#X = df[train_cols]
y = df[label]

In [10]:
seed = 1
df_train, df_test, y_train, y_test = train_test_split(df, y, test_size=0.20, random_state=seed)

In [11]:
df_train.shape

(332, 19249)

In [12]:
df_test.shape

(83, 19249)

In [13]:
df_train.head()

,Unnamed: 0,MT-CO1,GFAP,MT-ND4,MT-CO3,MT-CO2,MT-ATP6,EEF1A1,MT-ND2,MT-CYB,MT-ND5,MT-ND1,MBP,CLU,ACTB,SPARC,ACTG1,VIM,GAPDH,PLP1,TUBA1A,MT-ND3,EEF2,MT-ND6,FTH1,PEA15,APOE,AQP4,MT-ND4L,BCAN,B2M,FTL,SPP1,TPT1,PSAP,RPL3,PTPRZ1,GNAS,RPLP0,TMSB4X,...,LYZL6,OR6Q1,PRAMEF13,AL353572.3,FAM236C,AC233724.17,AC018793.3,AC106774.8,AC093762.1,OR8B2,AL132671.2,AC233724.19,AC106774.6,AC233724.20,AC233724.21,USP17L13,USP17L17,AC092017.3,AC106774.9,AC005833.1,AC106774.7,OR4F4,PRAMEF6,DEFB103A,DEFB103B,PRAMEF9,MBD3L4,TSPY3,AC018793.2,AC018793.4,AC104581.3,BPY2,BPY2B,BPY2C,PRAMEF5,AC106774.5,AC083800.1,AC118470.1,target,HighLowGrade
159,BS_D29RPBSZ,452321.54,216161.0,199327.42,118267.33,74216.85,100153.45,54390.97,134324.54,126867.0,190687.99,124057.09,871.00,130985.0,86782.57,20818.00,45049.67,105224.00,55210.66,273.0,16883.94,25176.0,30160.0,3954.94,73015.30,8324.00,67453.0,5883.0,18123.0,8400.00,16548.0,36299.35,19503.0,13531.67,73163.0,11338.08,24195.0,14436.00,13047.63,19466.24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
95,BS_5CTVXVRX,181744.03,91.0,17946.99,135002.63,35015.35,10876.90,1025556.42,4998.03,16899.0,9001.00,8360.56,376.62,3421.0,166512.70,26949.00,128894.00,66706.00,94238.93,230.0,16677.53,7607.0,214644.0,515.96,14612.38,3251.00,4229.0,19.0,2849.0,6.00,31901.0,52365.00,4800.0,144409.72,29590.0,79327.58,91.0,18677.00,102232.05,78266.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
11,BS_PXCPK5XS,452627.50,17821.9,189167.00,181179.84,127303.85,120060.21,426570.13,104953.66,140298.0,127217.00,81493.95,587.40,49776.0,235700.00,213354.99,151559.00,151174.56,154350.54,17229.0,353171.14,26953.0,73325.0,41216.00,59743.52,42748.81,35458.0,3243.0,20022.0,250718.76,101698.0,62421.01,60214.0,73593.02,71926.0,59701.77,157142.0,81352.36,54563.07,101068.00,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
374,BS_YZ2Z1Q6Y,721220.89,377771.0,410082.00,205727.94,119338.94,144419.13,84671.03,169671.05,219170.0,341511.99,175031.57,18584.00,116344.0,46338.61,120781.00,21603.33,15499.00,16224.62,4525.0,5062.08,38309.0,39992.0,5380.95,30032.81,10979.00,53938.0,6104.0,32679.0,39070.00,5100.0,12375.67,3550.0,11176.07,25784.0,19435.47,27786.0,16015.00,6122.52,6556.84,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,357.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
165,BS_C7A2TYAC,524802.48,968914.0,219724.00,179642.96,93605.92,105965.81,237301.95,90879.39,123222.0,159822.00,92200.18,43642.00,190568.0,318150.53,110694.00,142571.38,142524.00,96885.50,317831.0,68879.39,22727.0,114339.0,3462.97,94304.82,51484.00,72173.0,68364.0,14843.0,126863.00,65064.0,71084.12,20469.0,38223.34,101059.0,41898.36,377140.0,44129.00,28885.53,35913.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [14]:
df_test.head()

,Unnamed: 0,MT-CO1,GFAP,MT-ND4,MT-CO3,MT-CO2,MT-ATP6,EEF1A1,MT-ND2,MT-CYB,MT-ND5,MT-ND1,MBP,CLU,ACTB,SPARC,ACTG1,VIM,GAPDH,PLP1,TUBA1A,MT-ND3,EEF2,MT-ND6,FTH1,PEA15,APOE,AQP4,MT-ND4L,BCAN,B2M,FTL,SPP1,TPT1,PSAP,RPL3,PTPRZ1,GNAS,RPLP0,TMSB4X,...,LYZL6,OR6Q1,PRAMEF13,AL353572.3,FAM236C,AC233724.17,AC018793.3,AC106774.8,AC093762.1,OR8B2,AL132671.2,AC233724.19,AC106774.6,AC233724.20,AC233724.21,USP17L13,USP17L17,AC092017.3,AC106774.9,AC005833.1,AC106774.7,OR4F4,PRAMEF6,DEFB103A,DEFB103B,PRAMEF9,MBD3L4,TSPY3,AC018793.2,AC018793.4,AC104581.3,BPY2,BPY2B,BPY2C,PRAMEF5,AC106774.5,AC083800.1,AC118470.1,target,HighLowGrade
388,BS_862NMAR7,757880.34,1431919.0,465653.84,304550.91,215685.74,179379.60,379905.19,227760.88,228292.0,139771.00,222134.37,44760.00,608440.0,386329.89,246661.0,261171.50,89902.0,120302.72,63218.0,204446.93,38375.0,273174.0,2500.95,104163.59,199026.0,487879.0,96811.0,17161.0,521205.0,10615.0,56900.84,7310.0,25956.72,141143.0,84999.14,168862.0,106342.0,85465.18,26190.00,...,1.0,1.0,1.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.4,0.4,0.4,0.0,0.0,0.0,0.4,2268.48,0.0,0.0,4.82,0.5,0.5,4.01,0.0,3.84,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1.0,1
102,BS_66HQ3E4Z,466915.21,1208314.0,217188.92,155404.86,88420.92,99484.73,228500.94,95876.87,117132.0,169402.00,119464.51,38553.00,353139.0,305845.75,217380.0,128487.97,72290.0,105567.55,136301.0,67094.12,23223.0,126766.0,2023.99,118768.76,61219.0,174480.0,87080.0,18672.0,199624.0,44120.0,54935.93,18120.0,34588.71,128710.0,39291.62,326166.0,60043.0,24936.51,24133.04,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,651.04,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0
187,BS_EFEZB0ZH,322791.13,1740657.0,147939.87,121399.91,55890.86,62786.72,213259.97,74431.98,91998.0,119159.00,84751.97,7386.00,681635.0,288555.92,496903.0,115270.22,108977.0,97322.23,20107.0,56791.12,16784.0,132874.0,1437.97,135923.65,52559.0,171424.0,129058.0,9474.0,147210.0,51899.0,55890.64,11918.0,34261.99,162327.0,43816.21,163960.0,50911.0,25436.87,36453.90,...,0.0,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1162.86,0.0,0.0,0.67,0.0,0.0,0.00,0.0,0.00,0.0,2.0,0.0,0.0,0.0,0.0,0.33,0.0,0.0,0.0,0.0,0
162,BS_C51RB0YR,1150642.36,222780.0,499514.99,396129.86,267259.79,251604.89,296533.34,217611.39,363720.0,343012.00,198818.04,2165.00,1616980.0,410370.93,702442.0,126746.14,296294.0,287694.70,2610.0,198446.27,44271.0,212699.0,3294.93,89631.27,54910.0,258310.0,109155.0,32310.0,26171.0,65799.0,111115.06,58397.0,58407.71,190872.0,40652.31,198132.0,433718.0,50997.99,87188.51,...,0.0,0.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,112.44,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0
90,BS_52426AMF,656078.15,1845002.0,356146.08,203549.33,129971.56,169314.98,251682.86,179002.18,169410.0,284045.99,203193.26,45588.73,535311.0,286209.12,273309.0,115830.29,156027.0,96256.34,49888.0,76799.29,42208.0,103802.0,3427.96,99365.21,67274.0,105932.0,160462.0,30840.0,42011.0,72635.0,101345.17,23671.0,57154.78,120468.0,42751.99,214955.0,51517.0,25228.11,44645.11,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0


In [15]:
X_train = df_train[train_cols]
X_test = df_test[train_cols]

In [16]:
X_train.head()

,MT-CO1,GFAP,MT-ND4,MT-CO3,MT-CO2,MT-ATP6,EEF1A1,MT-ND2,MT-CYB,MT-ND5,MT-ND1,MBP,CLU,ACTB,SPARC,ACTG1,VIM,GAPDH,PLP1,TUBA1A,MT-ND3,EEF2,MT-ND6,FTH1,PEA15,APOE,AQP4,MT-ND4L,BCAN,B2M,FTL,SPP1,TPT1,PSAP,RPL3,PTPRZ1,GNAS,RPLP0,TMSB4X,SPARCL1,...,MTRNR2L11,AC026461.4,LYZL6,OR6Q1,PRAMEF13,AL353572.3,FAM236C,AC233724.17,AC018793.3,AC106774.8,AC093762.1,OR8B2,AL132671.2,AC233724.19,AC106774.6,AC233724.20,AC233724.21,USP17L13,USP17L17,AC092017.3,AC106774.9,AC005833.1,AC106774.7,OR4F4,PRAMEF6,DEFB103A,DEFB103B,PRAMEF9,MBD3L4,TSPY3,AC018793.2,AC018793.4,AC104581.3,BPY2,BPY2B,BPY2C,PRAMEF5,AC106774.5,AC083800.1,AC118470.1
159,452321.54,216161.0,199327.42,118267.33,74216.85,100153.45,54390.97,134324.54,126867.0,190687.99,124057.09,871.00,130985.0,86782.57,20818.00,45049.67,105224.00,55210.66,273.0,16883.94,25176.0,30160.0,3954.94,73015.30,8324.00,67453.0,5883.0,18123.0,8400.00,16548.0,36299.35,19503.0,13531.67,73163.0,11338.08,24195.0,14436.00,13047.63,19466.24,38670.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,181744.03,91.0,17946.99,135002.63,35015.35,10876.90,1025556.42,4998.03,16899.0,9001.00,8360.56,376.62,3421.0,166512.70,26949.00,128894.00,66706.00,94238.93,230.0,16677.53,7607.0,214644.0,515.96,14612.38,3251.00,4229.0,19.0,2849.0,6.00,31901.0,52365.00,4800.0,144409.72,29590.0,79327.58,91.0,18677.00,102232.05,78266.99,5014.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,452627.50,17821.9,189167.00,181179.84,127303.85,120060.21,426570.13,104953.66,140298.0,127217.00,81493.95,587.40,49776.0,235700.00,213354.99,151559.00,151174.56,154350.54,17229.0,353171.14,26953.0,73325.0,41216.00,59743.52,42748.81,35458.0,3243.0,20022.0,250718.76,101698.0,62421.01,60214.0,73593.02,71926.0,59701.77,157142.0,81352.36,54563.07,101068.00,70884.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
374,721220.89,377771.0,410082.00,205727.94,119338.94,144419.13,84671.03,169671.05,219170.0,341511.99,175031.57,18584.00,116344.0,46338.61,120781.00,21603.33,15499.00,16224.62,4525.0,5062.08,38309.0,39992.0,5380.95,30032.81,10979.00,53938.0,6104.0,32679.0,39070.00,5100.0,12375.67,3550.0,11176.07,25784.0,19435.47,27786.0,16015.00,6122.52,6556.84,21241.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,357.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,524802.48,968914.0,219724.00,179642.96,93605.92,105965.81,237301.95,90879.39,123222.0,159822.00,92200.18,43642.00,190568.0,318150.53,110694.00,142571.38,142524.00,96885.50,317831.0,68879.39,22727.0,114339.0,3462.97,94304.82,51484.00,72173.0,68364.0,14843.0,126863.00,65064.0,71084.12,20469.0,38223.34,101059.0,41898.36,377140.0,44129.00,28885.53,35913.59,60257.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
X_test.head()

,MT-CO1,GFAP,MT-ND4,MT-CO3,MT-CO2,MT-ATP6,EEF1A1,MT-ND2,MT-CYB,MT-ND5,MT-ND1,MBP,CLU,ACTB,SPARC,ACTG1,VIM,GAPDH,PLP1,TUBA1A,MT-ND3,EEF2,MT-ND6,FTH1,PEA15,APOE,AQP4,MT-ND4L,BCAN,B2M,FTL,SPP1,TPT1,PSAP,RPL3,PTPRZ1,GNAS,RPLP0,TMSB4X,SPARCL1,...,MTRNR2L11,AC026461.4,LYZL6,OR6Q1,PRAMEF13,AL353572.3,FAM236C,AC233724.17,AC018793.3,AC106774.8,AC093762.1,OR8B2,AL132671.2,AC233724.19,AC106774.6,AC233724.20,AC233724.21,USP17L13,USP17L17,AC092017.3,AC106774.9,AC005833.1,AC106774.7,OR4F4,PRAMEF6,DEFB103A,DEFB103B,PRAMEF9,MBD3L4,TSPY3,AC018793.2,AC018793.4,AC104581.3,BPY2,BPY2B,BPY2C,PRAMEF5,AC106774.5,AC083800.1,AC118470.1
388,757880.34,1431919.0,465653.84,304550.91,215685.74,179379.60,379905.19,227760.88,228292.0,139771.00,222134.37,44760.00,608440.0,386329.89,246661.0,261171.50,89902.0,120302.72,63218.0,204446.93,38375.0,273174.0,2500.95,104163.59,199026.0,487879.0,96811.0,17161.0,521205.0,10615.0,56900.84,7310.0,25956.72,141143.0,84999.14,168862.0,106342.0,85465.18,26190.00,60102.0,...,0.0,0.0,1.0,1.0,1.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.4,0.4,0.4,0.0,0.0,0.0,0.4,2268.48,0.0,0.0,4.82,0.5,0.5,4.01,0.0,3.84,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
102,466915.21,1208314.0,217188.92,155404.86,88420.92,99484.73,228500.94,95876.87,117132.0,169402.00,119464.51,38553.00,353139.0,305845.75,217380.0,128487.97,72290.0,105567.55,136301.0,67094.12,23223.0,126766.0,2023.99,118768.76,61219.0,174480.0,87080.0,18672.0,199624.0,44120.0,54935.93,18120.0,34588.71,128710.0,39291.62,326166.0,60043.0,24936.51,24133.04,109691.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,651.04,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
187,322791.13,1740657.0,147939.87,121399.91,55890.86,62786.72,213259.97,74431.98,91998.0,119159.00,84751.97,7386.00,681635.0,288555.92,496903.0,115270.22,108977.0,97322.23,20107.0,56791.12,16784.0,132874.0,1437.97,135923.65,52559.0,171424.0,129058.0,9474.0,147210.0,51899.0,55890.64,11918.0,34261.99,162327.0,43816.21,163960.0,50911.0,25436.87,36453.90,127522.0,...,0.0,0.0,0.0,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1162.86,0.0,0.0,0.67,0.0,0.0,0.00,0.0,0.00,0.0,2.0,0.0,0.0,0.0,0.0,0.33,0.0,0.0,0.0
162,1150642.36,222780.0,499514.99,396129.86,267259.79,251604.89,296533.34,217611.39,363720.0,343012.00,198818.04,2165.00,1616980.0,410370.93,702442.0,126746.14,296294.0,287694.70,2610.0,198446.27,44271.0,212699.0,3294.93,89631.27,54910.0,258310.0,109155.0,32310.0,26171.0,65799.0,111115.06,58397.0,58407.71,190872.0,40652.31,198132.0,433718.0,50997.99,87188.51,177143.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,112.44,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
90,656078.15,1845002.0,356146.08,203549.33,129971.56,169314.98,251682.86,179002.18,169410.0,284045.99,203193.26,45588.73,535311.0,286209.12,273309.0,115830.29,156027.0,96256.34,49888.0,76799.29,42208.0,103802.0,3427.96,99365.21,67274.0,105932.0,160462.0,30840.0,42011.0,72635.0,101345.17,23671.0,57154.78,120468.0,42751.99,214955.0,51517.0,25228.11,44645.11,82273.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [18]:
y

0      1
1      1
2      1
3      1
4      1
      ..
410    1
411    1
412    1
413    1
414    1
Name: HighLowGrade, Length: 415, dtype: int64

In [19]:
import xgboost as xgb

In [20]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

In [21]:
regressor.fit(X_train, y_train)

[21:37:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [22]:
features = pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns=X_train.columns)

In [23]:
features.head()

,MT-CO1,GFAP,MT-ND4,MT-CO3,MT-CO2,MT-ATP6,EEF1A1,MT-ND2,MT-CYB,MT-ND5,MT-ND1,MBP,CLU,ACTB,SPARC,ACTG1,VIM,GAPDH,PLP1,TUBA1A,MT-ND3,EEF2,MT-ND6,FTH1,PEA15,APOE,AQP4,MT-ND4L,BCAN,B2M,FTL,SPP1,TPT1,PSAP,RPL3,PTPRZ1,GNAS,RPLP0,TMSB4X,SPARCL1,...,MTRNR2L11,AC026461.4,LYZL6,OR6Q1,PRAMEF13,AL353572.3,FAM236C,AC233724.17,AC018793.3,AC106774.8,AC093762.1,OR8B2,AL132671.2,AC233724.19,AC106774.6,AC233724.20,AC233724.21,USP17L13,USP17L17,AC092017.3,AC106774.9,AC005833.1,AC106774.7,OR4F4,PRAMEF6,DEFB103A,DEFB103B,PRAMEF9,MBD3L4,TSPY3,AC018793.2,AC018793.4,AC104581.3,BPY2,BPY2B,BPY2C,PRAMEF5,AC106774.5,AC083800.1,AC118470.1
0,0.000025,0.000046,0.000469,2.099912e-07,0.0,0.0,0.003159,0.0,0.000033,0.0,0.0,0.000526,0.0,0.001345,0.000361,0.0,0.000004,0.0,0.000175,0.000375,0.000002,0.0,0.0,0.000005,0.0,0.002282,0.000018,0.0,0.000053,0.0,0.000002,0.000006,0.0,0.0,0.000032,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
y_pred = regressor.predict(X_test)

In [25]:
preds_series = pd.Series(y_pred)

In [26]:
preds_series.to_csv('y_test_reg_pred.csv')

In [27]:
from sklearn.metrics import mean_squared_error

In [28]:
mean_squared_error(y_test, y_pred)

0.04518411893223851

In [29]:
y_pred = regressor.predict(X_train)

In [30]:
preds_series = pd.Series(y_pred)

In [31]:
preds_series.to_csv('y_train_reg_pred.csv')

In [32]:
mean_squared_error(y_train, y_pred)

5.310466292408437e-05

In [33]:
features_T = features.transpose()

In [34]:
features_T.to_csv('features.csv')

In [35]:
xgb_cl = xgb.XGBClassifier()

In [36]:
xgb_cl.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
preds = xgb_cl.predict(X_test)

In [39]:
preds_series = pd.Series(preds)

In [40]:
preds_series.to_csv('y_test_pred.csv')

In [41]:
accuracy_score(y_test, preds)

0.9518072289156626

In [42]:
preds = xgb_cl.predict(X_train)

In [43]:
preds_series = pd.Series(preds)

In [44]:
preds_series.to_csv('y_train_pred.csv')

In [45]:
accuracy_score(y_train, preds)

1.0

In [46]:
y_train.to_csv('y_train.csv')

In [47]:
y_test.to_csv('y_test.csv')

In [48]:
features_cls = pd.DataFrame(xgb_cl.feature_importances_.reshape(1, -1), columns=X_train.columns)

In [49]:
features_T = features_cls.transpose()

In [50]:
features_T.to_csv('features_cls.csv')